In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
513,I don't know how expensive was the creation of...,negative
577,I think you would have to be from the USA to g...,positive
383,1985 was a good year for films - maybe even gr...,positive
10,Now we know where they got the idea of Snakes ...,negative
403,Blackwater Valley Exorcism is set on a small t...,negative


In [6]:

def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [7]:
df = normalize_text(df)
df.head()

,review,sentiment
513,know expensive creation movie effect awful hal...,negative
577,think would usa get lot joke liked princess br...,positive
383,good year film maybe even great one missing go...,positive
10,know got idea snake plane put bluntly pay see ...,negative
403,blackwater valley exorcism set small town ranc...,negative


In [8]:
df['sentiment'].value_counts()

sentiment
negative    266
positive    234
Name: count, dtype: int64

In [9]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [10]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
513,know expensive creation movie effect awful hal...,0
577,think would usa get lot joke liked princess br...,1
383,good year film maybe even great one missing go...,1
10,know got idea snake plane put bluntly pay see ...,0
403,blackwater valley exorcism set small town ranc...,0


In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [20]:
vectorizer = CountVectorizer(max_features=150)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/faizahmedd890/ml-project.mlflow')
dagshub.init(repo_owner='faizahmedd890', repo_name='ml-project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "faizahmedd890/ml-project"

2026-02-08 22:57:34,045 - INFO - Initialized MLflow to track repo "faizahmedd890/ml-project"


Repository faizahmedd890/ml-project initialized!

2026-02-08 22:57:34,060 - INFO - Repository faizahmedd890/ml-project initialized!


<Experiment: artifact_location='mlflow-artifacts:/a1f75ebc488e4717b4d08ff4aa2731c6', creation_time=1770571234153, experiment_id='0', last_update_time=1770571234153, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000) 

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

      
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-02-08 22:57:37,525 - INFO - Starting MLflow run...


2026-02-08 22:57:38,111 - INFO - Logging preprocessing parameters...
2026-02-08 22:57:39,303 - INFO - Initializing Logistic Regression model...
2026-02-08 22:57:39,307 - INFO - Fitting the model...
2026-02-08 22:57:39,434 - INFO - Model training complete.
2026-02-08 22:57:39,440 - INFO - Logging model parameters...
2026-02-08 22:57:39,812 - INFO - Making predictions...
2026-02-08 22:57:39,812 - INFO - Calculating evaluation metrics...
2026-02-08 22:57:39,843 - INFO - Logging evaluation metrics...
2026-02-08 22:57:42,557 - INFO - Saving and logging the model...
c:\Users\faiza\Downloads\ml-project\mlflow\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or ma